In [568]:
import yaml
from coffeetools import coffee
import IPython, jinja2

In [569]:
IPython.display.Javascript(coffee.compile("""
window.require.config
    paths: 
        d3: '//d3js.org/d3.v3.min'
        jquery: '/static/components/jquery/dist/jquery.min'
        jqueryui: 'https://cdn.jsdelivr.net/jquery.ui/1.11.4/jquery-ui.min'
        baobab: 'https://cdn.rawgit.com/Yomguithereal/baobab/master/build/baobab'
    shim: 
        jqueryui: 
            exports: '$'
            deps: ['jquery'] 
"""))

<IPython.core.display.Javascript object>

In [570]:
from bokeh.sampledata import iris
import json

In [571]:
df=iris.flowers
payload = df.to_dict(orient='split')
payload['index'] = [float(v) for v in payload['index']]
#payload

In [572]:
IPython.display.HTML("""
<div class="vs"></div>
<ul class='buttons'>
<li><a id="shuffle">shuffle</a></li>
</ul>
""")
# SCROLL ON THE RIGHT SIDE OF THE CYAN BOX

In [581]:
IPython.display.Javascript(coffee.compile(jinja2.Template("""
require ['d3','baobab'], (d3,Baobab)->
    d3.longscroll = (Selection)->
        max_rows = Selection.tree.get 'table','max_rows'
        size = Selection.tree.get('index').length
        
        s = (v)->"#{v}px"

        rowHeight = Selection.selection.selectAll('tr.values')[0].map (node)=> node.offsetHeight
        height = Selection.selection.property 'offsetHeight'
        offset = d3.scale.linear()
            .domain [0,size]
            .range [0,height*d3.mean(rowHeight)] # Only use the value rows 

        Selection.selection.selectAll('.longscroll').remove()

        before = Selection.selection.selectAll('div.before').data([0])
        before.enter().insert('div',':first-child').classed {'before':yes,'longscroll':yes}

        current = Selection.selection.selectAll('div.after').data([0])
        current.enter().append('div').classed {'current':yes,'longscroll':yes}

        after = Selection.selection.selectAll('div.after').data([0])
        after.enter().append('div').classed({'after':yes,'longscroll':yes})

        Selection.selection.on 'scroll.longscroll', ()-> 
            position = Math.floor offset.invert @scrollTop
            scroll @scrollTop,position
        scroll_width = 50
        updateSize = ()->
            table = Selection.selection.selectAll 'table'
                .style
                    position: 'absolute'
                    left: s Selection.selection.node().offsetLeft
                    top: s Selection.selection.node().offsetTop
                    'margin-top': s 0
            Selection.selection.style 
                'overflow-y': 'auto'              
                height: s table.property 'offsetHeight'
                width: s scroll_width+table.property 'offsetWidth'
                'background-color': 'cyan'            
        Selection.selection.select '::-webkit-scrollbar','width',s scroll_width
        
        scroll = (scrollTop,position)->
            current.each ()->
                updateSize()
                position = d3.max [0, d3.min [size - max_rows, position]]
                before.style("height", s offset position )
                after.style("height", s offset size-position)
                if Math.abs(position - Selection.tree.get('table','iloc'))>=1
                    Selection.tree.select('table','iloc').set position
        Selection.selection.node().click()
        scroll 0, Selection.tree.get('table','iloc')
        updateSize()
        
    class window.DataSource 
        ### 
        A data source manages raw and derived data for different selections.  A data source can 
        have many selections.  The raw data is expected to be in a split format
        
        data: mxn
        columns: n x 1
        index: m x 1
        ###
        
        constructor: (@data)->
            @tree = new Baobab @data
            if @tree.select('columns').exists()
                for column,column_index in columns = @get 'columns' 
                    ### Create Dynamic Nodes for Each Column Data Source ###
                    @addColumnDataSource column
            if @tree.select('index').exists()    
                ### Create a data source for the index ###
                @addColumnDataSource 'index', Baobab.monkey ['index'], (data)-> data
            
        updateIndex: (new_index)->
            ### Set sorted data and indices ###
            @set 'data', d3.permute @get('data'), new_index.map (value)=> 
                    @ColumnDataSource('index').indexOf value
            @set 'index', new_index
            this
            
        addDerivedDataSource: (name, columns, f )->
            ### return the new data source ###
            cursor = @tree.select('columns')
            cursor.push name
            @addColumnDataSource name, Baobab.monkey columns.map((value)=>['ColumnDataSource',value,'data'])..., f
            data = cursor.get().map((column_name)=> @ColumnDataSource column_name)
            @tree.set 'data', d3.zip data...
            @ColumnDataSource name
            
        ColumnDataSource: (column_name)->
            if Array.isArray column_name
                d3.zip column_name.map((c)=>@get('ColumnDataSource', c,'data'))...
            else
                Array @get('ColumnDataSource', column_name,'data')...
                
        addColumnDataSource: (column, monkey)->
            monkey ?= Baobab.monkey ['data'], ['.','name'], ['columns'], (data,name,columns)-> 
                column_index = columns.indexOf name
                data.map (value)=> value[column_index]
            @set ['ColumnDataSource', column], 
                name: column
                data: monkey
                
        sort: (columns,direction='ascending')->
            ### Multisort on an  ###
            MultiSort = (a,b,direction='ascending',i=0)->
                ### Multisorting function in d3 ###
                [a[i],b[i]] = [parseFloat(a[i]),parseFloat(b[i])]
                if a[i] == b[i] then MultiSort a, b, direction, i+1
                d3[direction] a[i],b[i]

            columns = if not Array.isArray columns then [columns] else columns
            sorted = d3.zip columns.map((c)=> @ColumnDataSource c)..., @ColumnDataSource 'index'
                .sort (a,b)-> MultiSort a,b,direction
            @updateIndex sorted.map (value)-> value[columns.length]     
            this
            
        order: ()-> 
            ### Order the original indices ###
            @sort('index')
            
        shuffle: ()-> 
            ### Randomly Sort the Indices ###
            @updateIndex d3.shuffle @ColumnDataSource 'index'

        get: (args...)->  @tree.get args...
        set: (args...)->  @tree.set args...

    class Selection
        constructor: (@selection)->

        update: (selection,selector,data,direction='down')->
            ### Repeatable pattern for creating, updating, and removing data dependent dom elements ###
            [tag,classes...] = selector.split '.'
            selection = selection.selectAll(selector).data data
            if direction in ['right','down']
                selection.enter().append tag
            else if direction in ['left','up']
                selection.enter().insert tag, ':first-child'
            classes.forEach (c)=> selection.classed c, yes
            selection.exit().remove()
            selection

        clear: ()-> @selection.html ''

    class Table extends Selection
        constructor: (selection,@tree,config)->  
            
            super selection
            @tree.set ['table'],config
            
            @build()
        build: ->
            table_cursor = @tree.select 'table' 
            table = @update @selection, 'table', [1]
            order = table_cursor.get 'columns','order'
            heading = @update table, 'tr.heading', [order], 'up'
            _this = @
                
            heading.each (columns)->
                _this.update d3.select(@), 'th.index', order[0],'left'
                    .text (d)-> d
                _this.update d3.select(@), 'th.value', order[1],'right'
                    .text (d)-> d
            values = @update table, 'tr.values', table_cursor.get('columns','values'), 'down'
            values.each (row_values)->    
                _this.update d3.select(@), 'th.index', order[0],'left'
                _this.update d3.select(@), 'td.value', order[1],'right'
                d3.select(@).selectAll 'th.index,td.value'
                    .data row_values
                    .text (d)-> d
    class Manager
        ### 
        One Data Source for many selections
        Attaches event driven data tress to selections 
        ###
        constructor: (@data)->
            ### @data changes state many times ###
            @DataSource = new DataSource @data
            @Selections = []

        attach: (selection,Selection,config)->
            selections_list = @Selections.filter (s)=> 
                s.selection.node() == selection.node()
            if selections_list.length == 0
                id = @Selections.push new Selection selection, @DataSource.tree, config
                @Selections[id-1]
            else
                selections_list[0]


    ### Create the Manager for the Blaze payload ###
    window.manager = new Manager {{json.dumps(payload)}}
    
    ### Add a Selection to the Manager with ad hoc settings ###
    window.table = manager.attach d3.selectAll('.vs').html(''), Table,                 
        max_rows: 10
        iloc: 0
        show_id: yes
        index: Baobab.monkey ['.','max_rows'],['.','iloc'],['index'], (rows, iloc, index)-> 
            d3.range iloc, iloc+rows
                .map (iloc)=> index[iloc]
        scrolling:
            speed: 10 #rows at a time
        columns:
            index: ['index','species']
            exclude: ['sepal_length']
            order: Baobab.monkey ['.','index'],['.','exclude'],['columns'], (index,exclude,columns)->
                [
                    index
                    columns.filter (c)=> c not in index
                        .filter (c)=> c not in exclude
                ]
            values: Baobab.monkey ['ColumnDataSource'],['..','index'],['.','order'], (cds, index, order)->
                v = d3.merge order
                    .map (column)=> 
                        index.map (i)=> cds[column]['data'][i]
                d3.zip v...
    
    #### Shuffle the data ####
    d3.selectAll('#shuffle').on 'click', ()=> manager.DataSource.shuffle()

    ### Set up listeners on the tree ###
    manager.DataSource.tree.select ['table']
        .on 'update', ()-> table.build()
    
    #### Fix the table width ###
    table.selection.select('table')
        .style 'width', '600px'
    window.scroller = d3.longscroll table 
    
    console.log table.selection.node()
    manager.DataSource.addDerivedDataSource 'mean', ['sepal_length','sepal_width','petal_length','petal_width'], (args...)->
        d3.zip args...
            .map (row_value)=> d3.mean row_value
""").render(**globals())))

<IPython.core.display.Javascript object>

---

# STOP HERE #

---

In [ ]:
IPython.display.Javascript(coffee.compile(jinja2.Template("""
require ['d3','baobab'], (d3,Baobab)->
        d3.longscroll = (table)->
            
            maxRows =tree.get('table','rows','max')
            size = tree.get('shape',0)
            rowHeight = tree.set ['table','rows','height'], d3.max table.selectAll('tr.rows')[0].map (_this)-> _this.offsetHeight
            offset = d3.scale.linear().domain([0,size]).range [0,size*rowHeight]
            Selection.selection.selectAll('.longscroll').remove()
            before = parent.selectAll('div.before').data([0])
            before.enter().insert('div',':first-child').classed {'before':yes,'longscroll':yes}
            current = parent.selectAll('div.after').data([0])
            current.enter().append('div').classed {'current':yes,'longscroll':yes}
            after = parent.selectAll('div.after').data([0])
            after.enter().append('div').classed({'after':yes,'longscroll':yes})
            parent.on 'scroll.longscroll', ()-> 
                position= Math.floor offset.invert @scrollTop
                scroll @scrollTop

            scroll = (scrollTop)->
                current.each ()->
                    console.log scrollTop, @scrollTop
                    @scrollTop = scrollTop
                    position0 = d3.max [0, d3.min [size - maxRows, Math.floor offset.invert(scrollTop)]]
                    before.style("height", "#{offset position0}px")
                    after.style("height", "#{offset (size-position0)}px")
                    if Math.abs(position0 - tree.get('table','rows','start'))>=tree.get('table','rows','increment')
                        tree.select('table','rows','start').set position0
                        id = '%s'
                        s = Bokeh.Collections('ColumnDataSource').get(id).get('selected')
                        s['1d']['indices'] = d3.range 0, tree.get('table','rows','max')
                            .map (i)-> tree.get 'table','index', (i + tree.get('table','rows','start'))
                        Bokeh.Collections('ColumnDataSource').get(id).set('selected',s)
                        Bokeh.Collections('ColumnDataSource').get(id).trigger 'select'
            scroll(0)
        `longscroll.render = function(_) {return arguments.length ? (render = _, longscroll) : render;};
        longscroll.rowHeight = function(_) {return arguments.length ? (rowHeight = +_, longscroll) : rowHeight;};
        longscroll.position = function(_) {return arguments.length ? (position = +_, longscroll) : position;};
        longscroll.size = function(_) {return arguments.length ? (size = +_, longscroll) : size;};`
        longscroll
""").render(**globals())))

In [ ]:
scrolling=IPython.display.Javascript(coffee.compile("""
require ['baobab','d3'], (Baobab,d3)->
    d3.longscroll = (Selection)->
        max_rows = Selection.tree.get 'table','max_rows'
        size = Selection.tree.get('index').length
        s = (v)->"#{v}px"

        rowHeight = Selection.selection.selectAll('tr.values')[0].map (node)=> node.offsetHeight
        height = Selection.selection.property 'offsetHeight'
        offset = d3.scale.linear()
            .domain [0,size]
            .range [0,height*d3.mean(rowHeight)/max_rows/Selection.tree.get('table','scrolling','speed')] # Only use the value rows 

        Selection.selection.selectAll('.longscroll').remove()

        before = Selection.selection.selectAll('div.before').data([0])
        before.enter().insert('div',':first-child').classed {'before':yes,'longscroll':yes}

        current = Selection.selection.selectAll('div.after').data([0])
        current.enter().append('div').classed {'current':yes,'longscroll':yes}

        after = Selection.selection.selectAll('div.after').data([0])
        after.enter().append('div').classed({'after':yes,'longscroll':yes})

        Selection.selection.on 'scroll.longscroll', ()-> 
            position = Math.floor offset.invert @scrollTop
            scroll @scrollTop,position
        scroll_width = 50
        updateSize = ()->
            table = Selection.selection.selectAll 'table'
                .style
                    position: 'absolute'
                    left: s Selection.selection.node().offsetLeft
                    top: s Selection.selection.node().offsetTop
                    'margin-top': s 0
            Selection.selection.style 
                'overflow-y': 'auto'              
                height: s table.property 'offsetHeight'
                width: s scroll_width+table.property 'offsetWidth'
                'background-color': 'cyan'            
        Selection.selection.select '::-webkit-scrollbar','width',s scroll_width
        
        scroll = (scrollTop,position)->
            current.each ()->
                updateSize()
                position = d3.max [0, d3.min [size - max_rows, position]]
                before.style("height", s offset position )
                after.style("height", s offset size-position)
                if Math.abs(position - Selection.tree.get('table','iloc'))>=1
                    Selection.tree.select('table','iloc').set position
        Selection.selection.node().click()
        scroll 0, Selection.tree.get('table','iloc')
        updateSize()
"""))
#IPython.display.display(scrolling, manager)

In [ ]:
scrolling=IPython.display.Javascript(coffee.compile("""
require ['baobab','d3'], (Baobab,d3)->
    d3.longscroll = ()->

        longscroll = (table)->
            maxRows =tree.get('table','rows','max')
            size = tree.get('shape',0)
            rowHeight = tree.set ['table','rows','height'], d3.max table.selectAll('tr.rows')[0].map (_this)-> _this.offsetHeight
            
            offset = d3.scale.linear()
                .domain [0,size]
                .range [0,size*rowHeight]

            Selection.selection.selectAll('.longscroll').remove()

            before = parent.selectAll('div.before').data([0])
            before.enter().insert('div',':first-child').classed {'before':yes,'longscroll':yes}

            current = parent.selectAll('div.after').data([0])
            current.enter().append('div').classed {'current':yes,'longscroll':yes}

            after = parent.selectAll('div.after').data([0])
            after.enter().append('div').classed({'after':yes,'longscroll':yes})

            parent.on 'scroll.longscroll', ()-> 
                position= Math.floor offset.invert @scrollTop
                scroll @scrollTop

            scroll = (scrollTop)->
                current.each ()->
                    console.log scrollTop, @scrollTop
                    @scrollTop = scrollTop
                    position0 = d3.max [0, d3.min [size - maxRows, Math.floor offset.invert(scrollTop)]]
                    before.style("height", "#{offset position0}px")
                    after.style("height", "#{offset (size-position0)}px")
                    if Math.abs(position0 - tree.get('table','rows','start'))>=tree.get('table','rows','increment')
                        tree.select('table','rows','start').set position0
                        id = '%s'
                        s = Bokeh.Collections('ColumnDataSource').get(id).get('selected')
                        s['1d']['indices'] = d3.range 0, tree.get('table','rows','max')
                            .map (i)-> tree.get 'table','index', (i + tree.get('table','rows','start'))
                        Bokeh.Collections('ColumnDataSource').get(id).set('selected',s)
                        Bokeh.Collections('ColumnDataSource').get(id).trigger 'select'
            scroll(0)

        `longscroll.render = function(_) {
        return arguments.length ? (render = _, longscroll) : render;
        };
        longscroll.rowHeight = function(_) {
        return arguments.length ? (rowHeight = +_, longscroll) : rowHeight;
        };
        longscroll.position = function(_) {
        return arguments.length ? (position = +_, longscroll) : position;
        };
        longscroll.size = function(_) {
        return arguments.length ? (size = +_, longscroll) : size;
        };`
        longscroll

    table = d3.selectAll '#viewport table'
        .call d3.longscroll()

    #$(table.node()).colResizable {liveDrag:yes,fixed:yes}
    #d3.select('.JCLRgrips').style {'position':'absolute','left':0,'top':0}


"""%source.ref['id']))

In [ ]:
print(coffee.compile("""
###
The Table class creates virtual scrolling HTML tables using d3 selections
###
class TableSelection
    ### 
    Add columns to a table  
    Columns contain headers, info, data for either an index or data
    ###
    constructor: (@selection,@selector,@direction='right',@escaped=no)->
        ids = @selector.split('.')
        @tag = ids[0]
        @classes  = if @tag.length > 0 then ids[1..] else []
        @escaped = if @escaped then 'html' else 'text'
        
    clear: ()-> @selection.html ''
    
    create: (data,render=no)->
        selection = @selection.selectAll @selector
            .data data
        if @direction in ['down','right']
            selection.enter().append @tag
        else if @direction in ['up','left']
            selection.enter().insert @tag, ':first-child'
        selection.classed c, true for c in @classes
        selection.exit().remove()   
        selection
        
    render: (node)-> 
        d3.select(node)[@escaped] (d)-> d
        
    addCells: (selection, selector, direction)->
        ids = selector.split('.')
        tag = ids[0]
        classes  = if @tag.length > 0 then ids[1..] else []
        
        selection.each (d)->
            console.log tag
            cell = d3.select @
                .selectAll tag
                .filter ()-> classes.reduce ((p,n)=> p and d3.select(@).classed(n)), true
                .data d
            if direction in ['left','up']
                cell.enter().insert tag, ':first-child'
            else if direction in ['right','down']
                cell.enter().append tag
            
            for c in classes 
                cell.classed c, yes
            cell.text (d)-> d
            cell.exit().remove()

class Table extends TableSelection
    constructor: (@selection,@data)->
        @data['index'] ?= []
        @data['columns'] = @data['columns'].filter (d)=> d not in @data['index']
        @data['length'] = d3.merge([@data['index'],@data['columns']])[0].length
        @table = @build_table()
    build_table: ->
        @selection = @selection.selectAll 'table'
            .data [1] #table
        @selection.enter().append('table').classed('magic',yes)

        heading = @selection.selectAll 'tr.heading'
            .data [1] #heading
        heading.enter().append('tr').classed('heading',yes)
        heading.exit().remove()
        
        heading_value = heading.selectAll 'th.value'
            .data @data['columns']
        heading_value.enter().append('th').classed('value',yes)
        heading_value.exit().remove()

        heading_index = heading.selectAll 'th.index'
            .data @data['index']
        heading_index.enter().append('th').classed('index',yes)
        heading_index.exit().remove()
        
        body = @selection.selectAll 'tr.data'
            .data d3.range(@data['length'])
        body.enter().append('tr').classed('data',yes)
        body.exit().remove()
        
        _data = @data
        body.each (i)->
            s = d3.select(@).selectAll 'th.index'
                .data d3.zip _data['index'].map((k)=>_data['data'][k])...
            s.enter().insert('th',':first-child').classed('index',yes)
            s.text (d)-> d
            s.exit().remove()

            s.selectAll 'td.value'
                .data d3.zip _data['index'].map((k)=>_data['data'][k])...
            s.enter().append('td').classed('value',yes)
            s.text (d)-> d
            s.exit().remove()




table = new Table d3.selectAll('#aa'), {columns:['x','y','z'],index:['x'],data: {x:[3,8,7],y:[1,8,7],z:[0,8,7]} }
#table.clear()
console.log table.selection.node()
""",bare=True))

In [ ]:
class CoffeeLoader(yaml.SafeLoader):
    pass

def coffee_constructor(loader, node):
    """
    Use pandas IO tools to easily load local and remote data files
    """
    bits = loader.construct_mapping(node, deep=True)
    print(bits.values())
    return """!!js/function\n"""+'\n'.join(['     '+l for l in coffee.compile(bits['monkey'],bare=True).split('\n')])
CoffeeLoader.add_constructor("!coffee", coffee_constructor)

In [ ]:
IPython.display.Javascript(coffee.compile("""
class a extends null

console.log extend {a:1},{'b':2}
""",bare=True))

In [ ]:
manager = IPython.display.Javascript(coffee.compile(jinja2.Template("""
require ['d3','baobab'], (d3,Baobab)->

    class Selection
        constructor: (@selection)->

        update: (selection,selector,data,direction='down')->
            ### Repeatable pattern for creating, updating, and removing data dependent dom elements ###
            [tag,classes...] = selector.split '.'
            selection = selection.selectAll(selector).data data
            if direction in ['right','down']
                selection.enter().append tag
            else if direction in ['left','up']
                selection.enter().insert tag, ':first-child'
            classes.forEach (c)=> selection.classed c, yes
            selection.exit().remove()
            selection

        clear: ()-> @selection.html ''

    class Table extends Selection
        constructor: (selection,@tree,config)->  
            
            super selection
            @tree.set ['table'],config
            
            @build()
        build: ->
            table_cursor = @tree.select 'table' 
            table = @update @selection, 'table', [1]
            order = table_cursor.get 'columns','order'
            heading = @update table, 'tr.heading', [order], 'up'
            _this = @
            heading.each (columns)->
                _this.update d3.select(@), 'th.index', order[0],'left'
                    .text (d)-> d
                _this.update d3.select(@), 'th.value', order[1],'right'
                    .text (d)-> d
            values = @update table, 'tr.values', table_cursor.get('columns','values'), 'down'
            values.each (row_values)->
                _this.update d3.select(@), 'th.index', order[0],'left'
                _this.update d3.select(@), 'td.value', order[1],'right'
                d3.select(@).selectAll 'th.index,td.value'
                    .data row_values
                    .text (d)-> d

            
    class Manager
        ### 
        One Data Source for many selections
        Attaches event driven data tress to selections 
        ###
        constructor: (@data)->
            ### @data changes state many times ###
            @DataSource = new DataSource @data
            @Selections = []

        attach: (selection,Selection,config)->
            selections_list = @Selections.filter (s)=> 
                s.selection.node() == selection.node()
            if selections_list.length == 0
                id = @Selections.push new Selection selection, @DataSource.tree, config
                @Selections[id-1]
            else
                selections_list[0]

    window.manager = new Manager {{json.dumps(payload)}}
    window.table = manager.attach d3.selectAll('.vs').html(''), Table,                 
        max_rows: 10
        iloc: 0
        index: Baobab.monkey ['.','max_rows'],['.','iloc'],['index'], (rows, iloc, index)-> 
            d3.range iloc, iloc+rows
                .map (iloc)=> index[iloc]
        scrolling:
            speed: 1 #rows at a time
        columns:
            index: ['species']
            exclude: ['sepal_length']
            order: Baobab.monkey ['.','index'],['.','exclude'],['columns'], (index,exclude,columns)->
                [
                    index
                    columns.filter (c)=> c not in index
                        .filter (c)=> c not in exclude
                ]
            values: Baobab.monkey ['ColumnDataSource'],['..','index'],['.','order'], (cds, index, order)->
                v = d3.merge order
                    .map (column)=> 
                        index.map (i)=> cds[column]['data'][i]
                d3.zip v...
        
    manager.DataSource.tree.select ['table','iloc']
        .on 'update', ()-> table.build()
    manager.DataSource.tree.select ['index']
        .on 'update', ()-> table.build()
    
    table.selection.select('table')
        .style 'width', '600px'
    window.scroller = d3.longscroll table 
    
    manager.DataSource.addDerivedDataSource 'mean', ['sepal_length','sepal_width','petal_length','petal_width'], (args...)->
        d3.zip args...
            .map (row_value)=> d3.mean row_value
""").render(**globals())))

In [ ]:
IPython.display.Javascript(coffee.compile("""
###
The Table class creates virtual scrolling HTML tables using d3 selections
###
class Table
    constructor: (@parent,@data)->
        table = @parent.selectAll 'table'
            .data @data
        table.enter().append('table')
        table.each ()-> d3.select(@).call Row
        table.exit().remove()

    build: ()->
        
            
class Row
    constructor: (
        @table,
        @cell_tag='td', 
        @method='append',
        @classes=[],
    )->
        @update = ()=>
            row = @table.selectAll 'tr'
                .data @table
            row.enter().append 'tr'
            row.each (d)-> d3.select(@).call Cell
            row.exit().remove()

    
        
        
d3.select '#a'
    .call Table, [[1,2,3,4]]
""",bare=True))

In [ ]:
print(coffee.compile("""
table = new class Table
    constructor: (@el,@tree)->
table::rows = ()->
    @tree
table::scroll
table::hover
table::click
table::heading
table::index|
""",bare=True))

In [ ]:
from bokeh.sampledata import  iris

In [ ]:
import json

In [ ]:
data = iris.flowers.iloc[:10].to_dict(orient='split')
del data['index']

In [ ]:
IPython.display.HTML("""<div id='a'></div>""")

In [ ]:
print(coffee.compile(jinja2.Template("""
require [
    '/static/bower_components/d3/d3.min.js'
], (d3)->
    window.data = {{json.dumps(data)}}
    
    class Table
        constructor: (@node,@data)->
            
            @table = d3.select(@node).selectAll 'table'
                .data [0]
            @table.enter().append 'table'
            @rows()
            @cells()
            @table.exit().remove()
            
            
                
    Table::rows = (classes={})->
        debugger
        @rows = @table.selectAll('tr').data @data
        @rows.enter().append 'tr'
        @rows.exit().remove()
        
    Table::cells = (selection,row_data,tag='td',classed=[])->
        @rows.each (row_data)->
            cell = d3.select(@).selectAll tag
                .filter (d)-> classed.reduce ((p,n)-> p and d3.select(@).classed n), yes
                .data row_data
            cell.enter().append 'td'
            cell.text (value)-> value
            cell.exit().remove()
                    
    d3.selection::table = Table
    table = new Table '#a', data['data']
    
""").render(**globals())))

In [ ]:
data = yaml.load( """
code: !coffee
    cursors: ['data']
    monkey: 
        ()->
          console.log "#{d} microphone check"
a: 1
z: 8
""",Loader=yaml.BaseLoader)
print(yaml.dump(data,default_flow_style=False))

In [ ]:
coffee.compile("""()-> console.log 1""",bare=True).split('\n')[0].startswith('//')

In [ ]:
jsstring = """
""".join([l for l in yaml.dump(data,default_flow_style=False).replace('"!!',"!!").split('\n') if l.strip() != '"' and  l.strip().startswith('//')])
print(jsstring)

In [ ]:
print(coffee.compile(jinja2.Template("""
require [
    'http://esprima.org/esprima.js'
    'https://cdnjs.cloudflare.com/ajax/libs/js-yaml/3.4.6/js-yaml.min.js'
], (esprima,yaml)->
    debugger
    window.parsed = yaml.load \"\"\"{{jsstring}}\"\"\"
    
""").render(**globals())))